In [2]:
import pymongo
import pandas as pd
import re

client = pymongo.MongoClient("mongodb://localhost:27017/")

mydb = client["Vulns"]

teams_col = mydb["teams"]
assets_col = mydb["assets"]
export_col = mydb["export"]

from neo4j import GraphDatabase

uri = "bolt://localhost:7687"  
username = "neo4j"  
password = "" 

driver = GraphDatabase.driver(uri, auth=(username, password))

driver.verify_connectivity()

# MongoDB

Calcular todas as vulnerabilidades em máquinas de ACR > 9 e entender qual o time que possui maior quantidade de vulnerabilidades, separando por tipos de vulnerabilidade e total juntando todas.

In [19]:
result = export_col.aggregate([
    {
        '$match': {
            'asset.ACR': {
                '$gte': 9
            }
        }
    }, {
        '$unwind': '$asset.team'
    }, {
        '$group': {
            '_id': '$asset.team', 
            'Critical': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Critical'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'High': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'High'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Medium': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Medium'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Low': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Low'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Info': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Info'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Total': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'Critical': -1, 
            'High': -1, 
            'Medium': -1, 
            'Low': -1
        }
    }
])

list(result)

[{'_id': 'Morocco',
  'Critical': 5557,
  'High': 8278,
  'Medium': 13930,
  'Low': 13695,
  'Info': 289,
  'Total': 41749},
 {'_id': 'Bhutan',
  'Critical': 3115,
  'High': 4628,
  'Medium': 7747,
  'Low': 7628,
  'Info': 163,
  'Total': 23281},
 {'_id': 'Malaysia',
  'Critical': 3096,
  'High': 4466,
  'Medium': 7488,
  'Low': 7219,
  'Info': 155,
  'Total': 22424},
 {'_id': 'Kazakhstan',
  'Critical': 3082,
  'High': 4630,
  'Medium': 7738,
  'Low': 7630,
  'Info': 165,
  'Total': 23245},
 {'_id': 'Malta',
  'Critical': 2952,
  'High': 4163,
  'Medium': 7052,
  'Low': 7126,
  'Info': 159,
  'Total': 21452},
 {'_id': 'Mauritius',
  'Critical': 2946,
  'High': 4235,
  'Medium': 7235,
  'Low': 7158,
  'Info': 154,
  'Total': 21728},
 {'_id': 'Australia',
  'Critical': 2926,
  'High': 4383,
  'Medium': 7333,
  'Low': 7211,
  'Info': 149,
  'Total': 22002},
 {'_id': 'France',
  'Critical': 2901,
  'High': 4387,
  'Medium': 7226,
  'Low': 7227,
  'Info': 173,
  'Total': 21914},
 {'_id': '

In [14]:
times = client['Vulns']['teams'].aggregate([
    {
        '$match': {
            'members': {
                '$elemMatch': {
                    '$regex': re.compile(r"(?i)[aeiou]$")
                }
            }
        }
    }, {
        '$project': {
            '_id': 0, 
            'name': 1
        }
    }
])

times = [x["name"] for x in list(times)]

result2 = client['Vulns']['export'].aggregate([
    {
        '$match': {
            '$and': [
                {
                    'vuln.pub_date': re.compile(r"^2012")
                }, {
                    'asset.ip': re.compile(r"^10")
                }, {
                    'asset.team': {
                        '$in': times
                    }
                }
            ]
        }
    }, {
        '$group': {
            '_id': '$asset.ip', 
            'total': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'total': -1
        }
    }
])

list(result2)

[{'_id': '10.150.152.142', 'total': 12},
 {'_id': '10.88.190.69', 'total': 12},
 {'_id': '10.13.91.60', 'total': 11},
 {'_id': '10.197.84.173', 'total': 10},
 {'_id': '10.159.213.67', 'total': 9},
 {'_id': '10.49.212.100', 'total': 9},
 {'_id': '10.56.212.0', 'total': 9},
 {'_id': '10.140.54.30', 'total': 9},
 {'_id': '10.213.192.206', 'total': 8},
 {'_id': '10.11.140.129', 'total': 8},
 {'_id': '10.200.41.123', 'total': 8},
 {'_id': '10.75.55.115', 'total': 8},
 {'_id': '10.116.51.69', 'total': 8},
 {'_id': '10.222.150.115', 'total': 8},
 {'_id': '10.233.135.147', 'total': 8},
 {'_id': '10.127.89.153', 'total': 8},
 {'_id': '10.35.189.13', 'total': 8},
 {'_id': '10.159.74.183', 'total': 7},
 {'_id': '10.109.156.40', 'total': 7},
 {'_id': '10.134.153.87', 'total': 7},
 {'_id': '10.152.214.148', 'total': 7},
 {'_id': '10.87.82.68', 'total': 7},
 {'_id': '10.205.15.193', 'total': 7},
 {'_id': '10.166.39.52', 'total': 7},
 {'_id': '10.84.253.119', 'total': 7},
 {'_id': '10.132.88.222', 't

# Neo4j

In [18]:
with driver.session() as session:
    query = "MATCH c=(s:Asset {ip: $ip})-[r:conectado_com*1..2]->(vizinho:Asset) RETURN c "
    result3 = session.run(query, ip="172.17.87.9")
    
    for record in result3:
            path = record["c"]
            print("Path:", path)

Path: <Path start=<Node element_id='4:04b278b9-45f7-47a9-942f-37e2b778b323:585' labels=frozenset({'Asset'}) properties={'ACR': 4.1, 'hostname': 'a405b784-a30e-4045-855b-a6d304d99789', 'ip': '172.17.87.9'}> end=<Node element_id='4:04b278b9-45f7-47a9-942f-37e2b778b323:11947' labels=frozenset({'Asset'}) properties={'ACR': 8.9, 'hostname': 'e8a80306-a2de-4d82-9bef-9727ed1cfa5a', 'ip': '192.168.252.233'}> size=1>
Path: <Path start=<Node element_id='4:04b278b9-45f7-47a9-942f-37e2b778b323:585' labels=frozenset({'Asset'}) properties={'ACR': 4.1, 'hostname': 'a405b784-a30e-4045-855b-a6d304d99789', 'ip': '172.17.87.9'}> end=<Node element_id='4:04b278b9-45f7-47a9-942f-37e2b778b323:12955' labels=frozenset({'Asset'}) properties={'ACR': 5.4, 'hostname': 'b526761c-1c60-4fc1-80c0-b1b9275f29df', 'ip': '10.232.224.8'}> size=2>
Path: <Path start=<Node element_id='4:04b278b9-45f7-47a9-942f-37e2b778b323:585' labels=frozenset({'Asset'}) properties={'ACR': 4.1, 'hostname': 'a405b784-a30e-4045-855b-a6d304d997

# Conjunta

In [16]:

result5 = client['Vulns']['export'].aggregate([
    {
        '$group': {
            '_id': '$asset.ip', 
            'Critical': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Critical'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'High': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'High'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Medium': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Medium'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Low': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Low'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Info': {
                '$sum': {
                    '$cond': [
                        {
                            '$eq': [
                                '$vuln.severity', 'Info'
                            ]
                        }, 1, 0
                    ]
                }
            }, 
            'Total': {
                '$sum': 1
            }
        }
    }, {
        '$sort': {
            'Critical': -1, 
            'High': -1, 
            'Medium': -1, 
            'Low': -1
        }
     },
    {
        '$limit': 50
    }
])


result5 = list(result5)

ips = [x["_id"] for x in result5]

ips

['172.30.149.33',
 '10.106.46.21',
 '192.168.24.84',
 '192.168.156.164',
 '10.249.134.129',
 '172.21.18.206',
 '192.168.168.104',
 '10.43.214.135',
 '10.115.109.117',
 '10.98.161.237',
 '10.164.206.82',
 '10.39.135.202',
 '172.26.53.168',
 '192.168.72.80',
 '192.168.213.115',
 '172.20.175.77',
 '172.21.39.213',
 '192.168.219.123',
 '10.155.255.71',
 '172.21.36.235',
 '172.24.17.60',
 '192.168.194.148',
 '172.24.142.163',
 '10.160.70.85',
 '192.168.161.205',
 '10.188.167.110',
 '10.37.101.104',
 '192.168.166.133',
 '10.60.101.116',
 '10.123.71.150',
 '10.112.31.26',
 '172.30.241.109',
 '172.25.89.30',
 '172.22.60.54',
 '172.17.237.153',
 '10.93.26.200',
 '172.25.30.129',
 '172.24.83.245',
 '172.21.245.49',
 '172.24.4.39',
 '10.233.97.123',
 '172.23.210.156',
 '10.76.38.89',
 '172.29.83.157',
 '172.21.65.204',
 '10.130.118.114',
 '192.168.126.182',
 '192.168.85.174',
 '10.218.6.194',
 '10.3.230.236']

In [30]:

with driver.session() as session:
    all_records = []
    
    query = """
    WITH $ip_list AS ips
    UNWIND ips AS ip
    MATCH c=allShortestPaths((s:Asset {ip:ip})-[r:conectado_com*1..3]->(vizinho:Asset))
    WHERE vizinho.ACR >= 9
    WITH ip, c, [node IN nodes(c) WHERE node.ACR >= 9] AS critical_assets
    UNWIND critical_assets AS asset
    RETURN ip, count(asset) AS critical_assets_count
    """

    
    subquery = session.run(query, ip_list=ips)
    for record in subquery:
            all_records.append({"ip":record["ip"], "critical_assets_count":record["critical_assets_count"]})
    

pd.DataFrame(all_records)

,ip,critical_assets_count
0,172.30.149.33,24
1,10.106.46.21,30
2,192.168.24.84,9
3,192.168.156.164,30
4,10.249.134.129,44
5,172.21.18.206,28
6,192.168.168.104,24
7,10.43.214.135,30
8,10.115.109.117,16
9,10.98.161.237,22


In [45]:
df1 = pd.DataFrame(result5)
df2 = pd.DataFrame(all_records)

df = pd.merge(df1, df2, how="left", right_on="ip", left_on="_id").drop(["ip"],axis=1)

In [46]:
df["vuln_score"] = df["critical_assets_count"] * 0.4 + df["Critical"] * 0.3 + df["High"] * 0.2 + df["Low"] * 0.08 + df["Info"] * 0.02

In [47]:
df.sort_values(by=["vuln_score"],ascending=False, na_position='first')

,_id,Critical,High,Medium,Low,Info,Total,critical_assets_count,vuln_score
22,172.24.142.163,16,12,15,16,1,60,NaN,NaN
13,192.168.72.80,16,15,27,19,0,77,85.0,43.32
39,172.24.4.39,16,4,21,20,0,61,65.0,33.20
34,172.17.237.153,16,7,21,12,0,56,65.0,33.16
18,10.155.255.71,16,14,16,18,0,64,49.0,28.64
28,10.60.101.116,16,9,26,15,0,66,49.0,27.40
19,172.21.36.235,16,13,16,25,0,70,44.0,27.00
4,10.249.134.129,17,13,10,20,1,61,44.0,26.92
48,10.218.6.194,15,14,11,18,0,58,40.0,24.74
16,172.21.39.213,16,14,22,20,0,72,38.0,24.40
